In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, get_data_commitment_maps

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')
create_dummy(data_path, dummy_data_path)

In [5]:
scales = [6]
selected_columns = ['col_name']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [6]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x = data[0]
    return s.stdev(x)

error = 0.01
_, verifier_model = computation_to_model(computation, error)

verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path, verifier_model, verifier_model_path)

/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:172: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/utils.py:1703: UserWarning: The exported ONNX model failed ONNX shape inf

In [7]:
# Prover/ data owner side
_, prover_model = computation_to_model(computation, error)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

Using 8 columns for non-linearity table.
Using 8 columns for non-linearity table.
Using 8 columns for non-linearity table.


 <------------- Numerical Fidelity Report (input_scale: 6, param_scale: 6, scale_input_multiplier: 1) ------------->

+--------------+--------------+--------------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error   | median_error | max_error    | min_error | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+--------------+--------------+--------------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| 0.0009598732 | 0.0019197464 | 0.0019197464 | 0         | 0.0009598732   | 0.0019197464     | 0.0019197464  | 0             | 0.0000018427131    | 0.00006583472      | 0.0000

==== Generate & Calibrate Setting ====
scale:  [6]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":6,"param_scale":6,"scale_rebase_multiplier":1,"lookup_range":[-162602,162656],"logrows":19,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":14432,"total_assignments":9950,"total_const_size":2120,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,6],"model_input_scales":[6],"module_sizes":{"kzg":[],"poseidon":[14432,[1]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}}],"required_range_checks":[[-32,32]],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1709714355149}


In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 59.26989006996155 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 14.578125
==== Generating Proof ====
proof:  {'instances': [['c371c58876874ace4444c2f2683e0d7bb7ce264f49872aa6deab40635ed90f23', '0100000000000000000000000000000000000000000000000000000000000000', 'a503000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x2ea53767627a825d488d58596fc992c33ac4780a4d585c0ae3a2664fe513ce900c79099b797ba238d29133da21f2ff81a4eb0312d8084b2f2bbd725c54f5e0c32bcf23f22b94c1764f8bfc2c66b97768bb62e3e69f0e11ced087b0eb74370a9f20e0ff4f1bdb3c4f58a80940b7b2df8024be0100008a578925e93bcad8fd274a1009140f48bb12422b923c1f33e6e8ede158c081a1a21b4f42ce1caa64749de71c42fac312d374a9ee18c477933db3cba1dfc1dd95fdf8369adbc3b00bd82c0e261eee129ba354368a9bed9a1e7e290052e6a230f5c4cb94c136f76fdca2e357045f74e768829996e853d7a7e947dcba3d38d74b9b735025bde53143cfa5a0da29ef4bae4e2a3885f3a37bbf7a7dbcc39fea652a70092999d921d15b880be3341e1f0ffc2c3

In [9]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)
print("Verifier gets result:", res)

Verifier gets result: [14.578125]
